In [3]:
import json
from typing import List, Dict

def find_earliest_start(machine_schedule: List[Dict], machine_capacity: int, job_qubits: int, base_time_per_qubit: float) -> float:
    """Find the earliest time the job can run without exceeding capacity."""
    duration = job_qubits * base_time_per_qubit
    t = 0.0
    time_step = 1.0

    while True:
        # Check current load in interval [t, t+duration]
        load = 0
        for job in machine_schedule:
            if not (t + duration <= job['start'] or t >= job['end']):  # overlap
                load += job['qubits']

        if load + job_qubits <= machine_capacity:
            return t
        t += time_step

def schedule_jobs_parallel(job_capacities, machine_capacities, base_time_per_qubit=10.0):
    # Sort jobs in descending order of qubits
    sorted_jobs = sorted(job_capacities.items(), key=lambda x: -x[1])
    
    # Initialize per-machine schedules
    machines = {name: [] for name in machine_capacities}

    schedule = []

    for job_id, qubit in sorted_jobs:
        if qubit == 0:
            continue

        duration = qubit * base_time_per_qubit

        assigned = False
        for machine_name, capacity in machine_capacities.items():
            machine_schedule = machines[machine_name]
            start_time = find_earliest_start(machine_schedule, capacity, qubit, base_time_per_qubit)
            end_time = start_time + duration

            job_info = {
                "job": job_id,
                "qubits": qubit,
                "machine": machine_name,
                "capacity": capacity,
                "start": start_time,
                "end": end_time,
                "duration": duration
            }

            schedule.append(job_info)
            machine_schedule.append(job_info)
            assigned = True
            break

        if not assigned:
            raise RuntimeError(f"No machine can accommodate job {job_id} with {qubit} qubits.")

    return schedule

def save_schedule_to_json(schedule, filename):
    with open(filename, "w") as f:
        json.dump(schedule, f, indent=4)

# Example usage
if __name__ == "__main__":
    job_capacities = {'0': 0, '1': 1, '2': 1, '3': 1, '4': 1, '5': 1}
    machine_capacities = {'fake_belem': 5, 'fake_manila': 5}
    
    schedule = schedule_jobs_parallel(job_capacities, machine_capacities, base_time_per_qubit=10.0)
    save_schedule_to_json(schedule, "scheduled_jobs.json")
    print("✅ Schedule saved to scheduled_jobs.json")


✅ Schedule saved to scheduled_jobs.json
